In [6]:
import os
os.chdir("/Users/678094/Desktop/Goodreads")

print(os.getcwd())

C:\Users\678094\Desktop\Goodreads


In [7]:
import requests
import bs4
import re
import numpy as np
import pandas as pd

In [8]:
page1 = 'https://www.goodreads.com/review/list/27739335-max?page=1&print=true&shelf=read&view=table'
page2 = 'https://www.goodreads.com/review/list/27739335-max?page=2&print=true&shelf=read&view=table'
page3 = 'https://www.goodreads.com/review/list/27739335-max?page=3&print=true&shelf=read&view=table'
page4 = 'https://www.goodreads.com/review/list/27739335-max?page=4&print=true&shelf=read&view=table'
page5 = 'https://www.goodreads.com/review/list/27739335-max?page=5&print=true&shelf=read&view=table'

In [9]:
req1, req2, req3, req4, req5 = [requests.get(page)
                                for page in [page1, page2, page3, page4, page5]]

In [10]:
soup1, soup2, soup3, soup4, soup5 = [bs4.BeautifulSoup(page.text, "lxml")
                                    for page in [req1, req2, req3, req4, req5] ]

In [11]:
type(req1)

requests.models.Response

In [12]:
soup1

<html>
<head>
<title>
Max’s 'read' books on Goodreads (97 books)
</title>
<script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();
 </script>
<script type="text/javascript">
    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t:a.d()})}}function b(m,l,h,j,i){var k={m:m,f:l,l:h,c:""+j,err:i,fromOnError:1,args:arguments};c.ueLogError(k);return false}b.skipTrace=1;e.onerror=b;function f(){c.uex("ld")}if(e.addEventListener){e.addEventListener("load",f,false)}else{if(e.attachEvent){e.attachEvent("onload",f)}}a.tag=d("tag");a.log=d("log");a.reset=d("rst");c.ue_csm=c;c.ue=a;c.ueLogError=d("err");c.ues=d("ues");c.uet=d("uet");c.uex=d("uex");c.uet("ue")})(window);(function(e,d){var a=e.ue||{};function c(g){if(!g){return}var f=d.head||d.getElementsByTagName("head")[0]||d.documentElement,h=d.createElement("script");h.async="async"

I select the title tag in the code to retrieve the titles of the books I have read:

In [13]:
titlesAll = []
for soup in [soup1, soup2, soup3, soup4, soup5]:
    for a in soup.find_all('a', title = True): 
        if a.text: 
            titlesAll.append(a['title'])

In [14]:
len(titlesAll)

582

As long as  I get also the 'did not like it' etc titles, I need to select only titles starting with a capital letter or starting with a digit (to be sure to caputre 1984 as well).
I use the method .isuppercase or .isdigit to maintain only those titles.
I should get 97 titles, which is the number of the 'read' books in my Goodreads account.

In [15]:
titlesAll

['La ragazza di Bube',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Crash',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Flow My Tears, the Policeman Said',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Lost in Music',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Things The Grandchildren Should Know',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Cocaine Nights',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'Chronicle of a Death Foretold',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'The Taming of the Shrew',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it was amazing',
 'The Merchant of Venice',
 'did not like it',
 'it was ok',
 'liked it',
 'really liked it',
 'it wa

In [16]:
titlesRead = []
for word in titlesAll:
    if word[0].isupper() or word[0].isdigit():
        titlesRead.append(word)

In [17]:
len(titlesRead)

97

In [18]:
titlesRead

['La ragazza di Bube',
 'Crash',
 'Flow My Tears, the Policeman Said',
 'Lost in Music',
 'Things The Grandchildren Should Know',
 'Cocaine Nights',
 'Chronicle of a Death Foretold',
 'The Taming of the Shrew',
 'The Merchant of Venice',
 'Othello',
 "A Midsummer Night's Dream",
 'King Lear',
 'Hamlet',
 'Romeo and Juliet',
 'Savages (Savages #2)',
 'Favorite Poems Of Emily Dickinson',
 'Tales of Mystery and Madness',
 'The Raven and Other Poems',
 'Journey to the Center of the Earth (Extraordinary Voyages, #3)',
 'Twenty Thousand Leagues Under the Sea (Extraordinary Voyages, #6)',
 'Around the World in Eighty Days (Extraordinary Voyages, #11)',
 'Sukkwan Island',
 'The Bishop Murder Case (A Philo Vance Mystery #4)',
 'The Garden Murder Case (A Philo Vance Mystery #9)',
 'Pop. 1280 (Crime Masterworks)',
 'The Black Arrow',
 'Treasure Island',
 'The Lovely Bones',
 'Ivanhoe (Waverley Novels, #5)',
 'The Year of the Hare',
 '1984',
 'Moby-Dick or, The Whale',
 'Billy Budd, Sailor',
 "Tou

In order to retrieve the ratings I first select all the 'span' tag in the soup, and later select only the element of the deriving list that start with lowercase, so to exclude the dates, which are all uppercase.

In [19]:
ratingsScraped = []
for soup in [soup1, soup2, soup3, soup4, soup5]:
    for a in soup.find_all('span', title = True): 
        if a.text: 
            ratingsScraped.append(a['title'])

In [20]:
ratingsScraped

['liked it',
 'November 04, 2018',
 'liked it',
 'December 29, 2016',
 'it was amazing',
 'December 28, 2016',
 'liked it',
 'December 23, 2016',
 'really liked it',
 'December 23, 2016',
 'liked it',
 'February 02, 2015',
 'it was amazing',
 'April 21, 2014',
 'liked it',
 'March 15, 2014',
 'liked it',
 'March 15, 2014',
 'really liked it',
 'March 15, 2014',
 'really liked it',
 'March 15, 2014',
 'liked it',
 'March 15, 2014',
 'really liked it',
 'March 15, 2014',
 'really liked it',
 'March 15, 2014',
 'liked it',
 'March 14, 2014',
 'really liked it',
 'March 14, 2014',
 'really liked it',
 'March 14, 2014',
 'really liked it',
 'March 14, 2014',
 'really liked it',
 'March 14, 2014',
 'really liked it',
 'March 14, 2014',
 'liked it',
 'March 14, 2014',
 'Goodreads Author!',
 'liked it',
 'March 14, 2014',
 'liked it',
 'March 14, 2014',
 'liked it',
 'March 14, 2014',
 'it was amazing',
 'March 14, 2014',
 'liked it',
 'March 14, 2014',
 'liked it',
 'March 14, 2014',
 'really

In order to drop the dates and keep only the actual rating given, I select only the items that start with lower case.

In [21]:
ratingsGiven = []
for word in ratingsScraped:
    if word[0].islower():
        ratingsGiven.append(word)

In [22]:
len(ratingsGiven)

97

In [25]:
ratingsGiven

['liked it',
 'liked it',
 'it was amazing',
 'liked it',
 'really liked it',
 'liked it',
 'it was amazing',
 'liked it',
 'liked it',
 'really liked it',
 'really liked it',
 'liked it',
 'really liked it',
 'really liked it',
 'liked it',
 'really liked it',
 'really liked it',
 'really liked it',
 'really liked it',
 'really liked it',
 'liked it',
 'liked it',
 'liked it',
 'liked it',
 'it was amazing',
 'liked it',
 'liked it',
 'really liked it',
 'it was ok',
 'really liked it',
 'it was amazing',
 'liked it',
 'it was ok',
 'liked it',
 'really liked it',
 'liked it',
 'really liked it',
 'really liked it',
 'liked it',
 'liked it',
 'really liked it',
 'liked it',
 'it was amazing',
 'liked it',
 'liked it',
 'it was amazing',
 'really liked it',
 'it was amazing',
 'it was amazing',
 'really liked it',
 'really liked it',
 'it was amazing',
 'really liked it',
 'really liked it',
 'really liked it',
 'really liked it',
 'really liked it',
 'it was amazing',
 'liked it',
 'r

In [26]:
maxRatings = pd.DataFrame(
    {'title': titlesRead,
     'rating': ratingsGiven
    })

In [27]:
maxRatings.head(20)

,title,rating
0,La ragazza di Bube,liked it
1,Crash,liked it
2,"Flow My Tears, the Policeman Said",it was amazing
3,Lost in Music,liked it
4,Things The Grandchildren Should Know,really liked it
5,Cocaine Nights,liked it
6,Chronicle of a Death Foretold,it was amazing
7,The Taming of the Shrew,liked it
8,The Merchant of Venice,liked it
9,Othello,really liked it


In [28]:
pd.crosstab(index = maxRatings['rating'], columns = "count")

col_0,count
rating,
it was amazing,28
it was ok,3
liked it,29
really liked it,37


Now I need to reclassify the rating column, transforming each evaluation into the corresponding number.

In [29]:
maxRatings['rating'] = maxRatings['rating'].map({'did not like it': 5, 'it was ok': 2, 'liked it' : 3, 'really liked it' : 4, 'it was amazing' : 5})

In [30]:
maxRatings.head(20)

,title,rating
0,La ragazza di Bube,3
1,Crash,3
2,"Flow My Tears, the Policeman Said",5
3,Lost in Music,3
4,Things The Grandchildren Should Know,4
5,Cocaine Nights,3
6,Chronicle of a Death Foretold,5
7,The Taming of the Shrew,3
8,The Merchant of Venice,3
9,Othello,4


In [31]:
pd.crosstab(index = maxRatings['rating'], columns = "count")

col_0,count
rating,
2,3
3,29
4,37
5,28


In [32]:
maxRatings.to_csv('maxRatings.csv')